# Build and Train

Now we're ready to begin building our sentiment classifier and begin training. We will be building out what is essentially a *frame* around Bert, that will allow us to perform language classification. First, we can initialize the Bert model, which we will load as a pretrained model from transformers.

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained("bert-base-cased")

# we can view the model using the summary method
bert.summary()

Now we need to define the frame around Bert, we need:

* **Two** input layers (one for input IDs and one for attention mask).

* A post-bert dropout layer to reduce the likelihood of overfitting and improve generalization.

* Max pooling layer to convert the 3D tensors output by Bert to 2D.

* Final output activations using softmax for outputting categorical probabilities.

In [ ]:
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name="input_ids", dtype="int32")
mask = tf.keras.layers.Input(shape=(512,), name="attention_mask", dtype="int32")

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[
    1
]  # access final activations (alread max-pooled) [1]
# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(1024, activation="relu")(embeddings)
y = tf.keras.layers.Dense(5, activation="softmax", name="outputs")(x)

We can now define our model, specifying input and output layers. Finally, we *can* freeze the Bert layer because Bert is already highly trained, and contains a huge number of parameters so will take a very long time to train further. Nonetheless, if you'd like to train Bert too, there is nothing wrong with doing so.

In [ ]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
# model.layers[2].trainable = False

# print out model summary
model.summary()

Our model architecture is now setup, and we can initialize our training parameters like so:

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

Now all we need to do is train our model. For this, we need to load in our training and validation datasets - which also requires our dataset element specs to be defined.

In [ ]:
element_spec = (
    {
        "input_ids": tf.TensorSpec(shape=(16, 512), dtype=tf.float64, name=None),
        "attention_mask": tf.TensorSpec(shape=(16, 512), dtype=tf.float64, name=None),
    },
    tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None),
)

# load the training and validation sets
train_ds = tf.data.experimental.load("train", element_spec=element_spec)
val_ds = tf.data.experimental.load("val", element_spec=element_spec)

# view the input format
train_ds.take(1)

And now we train our model as usual.

In [ ]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Finally, we save our model!

In [ ]:
model.save("sentiment_model")